### Music Generator based on Genre using VanillaGAN

In [2]:
!pip install librosa

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/260.1 kB ? eta -:--:--
   ---------------------------------------- 260.1/260.1 kB 5.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/64.6 kB ? eta -:--:--
   ---------------------------------------- 64.6/64.6 kB ? eta 0:00:00
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ----------------- ---------------------- 0.4/1.0 MB 13.4 MB/s eta 0:00:01
   ---------------------------------------  1.0/1.0 MB 12.9 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 10.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/166.7 kB ? eta -:--:--
   --------------------------------------- 166.7/166.7 kB 10.4 MB/s eta 0:00:00


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [3]:
# importing necessary libraries

import os
import numpy as np
import librosa
import librosa.display
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')

In [8]:
# Path to dataset
DATA_PATH = r'E:\Deep Learning\Music Generator by Genre\GTZAN Data\genres_original'

# Define fixed length for spectrograms (e.g., 128 time frames)
FIXED_LENGTH = 1000

In [9]:
# Function to load and process Mel-Spectrograms for a specific genre
def load_data(genre, sr=22050, n_fft=2048, hop_length=512, n_mels=128):
    genre_path = os.path.join(DATA_PATH, genre)
    mel_spectrograms = []

    for file_name in os.listdir(genre_path):
        file_path = os.path.join(genre_path, file_name)
        if file_name.endswith('.wav'):
            y, _ = librosa.load(file_path, sr=sr)
            mel = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
            mel_db = librosa.power_to_db(mel, ref=np.max)

            # Fix the shape of spectrograms (padding or truncating)
            if mel_db.shape[1] < FIXED_LENGTH:
                # Pad with zeros if shorter than FIXED_LENGTH
                pad_width = FIXED_LENGTH - mel_db.shape[1]
                mel_db = np.pad(mel_db, ((0, 0), (0, pad_width)), mode='constant')
            else:
                # Truncate if longer than FIXED_LENGTH
                mel_db = mel_db[:, :FIXED_LENGTH]

            mel_spectrograms.append(mel_db)

    return np.array(mel_spectrograms)

In [10]:
def preprocess_data(mel_spectrograms):
    mel_spectrograms = (mel_spectrograms - mel_spectrograms.min()) / (mel_spectrograms.max() - mel_spectrograms.min())
    mel_spectrograms = mel_spectrograms.astype('float32')
    mel_spectrograms = mel_spectrograms[..., np.newaxis]
    return mel_spectrograms

In [11]:
# Load data for a specific genre
genre = "classical" 
mel_spectrograms = load_data(genre)
mel_spectrograms = preprocess_data(mel_spectrograms)

print(f'{mel_spectrograms[0]}')

print(f'Classical Genre Mel Spectrograms shape: {mel_spectrograms.shape}')

[[[0.56033975]
  [0.585518  ]
  [0.58630806]
  ...
  [0.5648533 ]
  [0.52628505]
  [0.5426846 ]]

 [[0.6440804 ]
  [0.6800323 ]
  [0.70510846]
  ...
  [0.65127456]
  [0.6460861 ]
  [0.63288176]]

 [[0.6583756 ]
  [0.716168  ]
  [0.7374484 ]
  ...
  [0.7215705 ]
  [0.75937396]
  [0.77298266]]

 ...

 [[0.        ]
  [0.        ]
  [0.        ]
  ...
  [0.        ]
  [0.        ]
  [0.        ]]

 [[0.        ]
  [0.        ]
  [0.        ]
  ...
  [0.        ]
  [0.        ]
  [0.        ]]

 [[0.        ]
  [0.        ]
  [0.        ]
  ...
  [0.        ]
  [0.        ]
  [0.        ]]]
Classical Genre Mel Spectrograms shape: (100, 128, 1000, 1)


In [12]:
# GAN Parameters
latent_dim = 100
input_shape = mel_spectrograms.shape[1:]

print(input_shape)

(128, 1000, 1)


In [13]:
# Building Generator Model

def build_generator(latent_dim):
    model = tf.keras.Sequential([

        # project noise vector to initial feature map
        layers.Dense(512 * 4 *4, input_dim=latent_dim),
        layers.Reshape((4,4,512)),

        # Upsampling layers with increasing resolution
        layers.Conv2DTranspose(256, kernel_size=5, strides=(2, 2), padding='same', activation='relu'),
        layers.BatchNormalization(),

        layers.Conv2DTranspose(128, kernel_size=5, strides=(2, 5), padding='same', activation='relu'),
        layers.BatchNormalization(),

        layers.Conv2DTranspose(64, kernel_size=5, strides=(2, 5), padding='same', activation='relu'),
        layers.BatchNormalization(),

        layers.Conv2DTranspose(32, kernel_size=3, strides=(2, 5), padding='same', activation='relu'),
        layers.BatchNormalization(),


        # Output layer (mel-spectrogram grayscale image)
        layers.Conv2DTranspose(1, kernel_size=5, strides=(2,1), padding='same', activation='tanh')

    ])

    return model

In [14]:
# Building Discriminator Model

def build_discriminator(input_shape):
    model = tf.keras.Sequential([
        # First convolutional layer
        layers.Conv2D(64, kernel_size=5, strides=2, padding='same', input_shape=input_shape),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.3),

        # Second convolutional layer
        layers.Conv2D(128, kernel_size=5, strides=2, padding='same'),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.3),

        # Third convolutional layer
        layers.Conv2D(256, kernel_size=5, strides=2, padding='same'),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.3),

        # Fourth convolutional layer
        layers.Conv2D(512, kernel_size=5, strides=2, padding='same'),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.3),

        layers.Flatten(),

        # Dense layers for classification
        layers.Dense(1024),
        layers.LeakyReLU(alpha=0.2),
        layers.Dropout(0.5),

        layers.Dense(1, activation='sigmoid')  # Binary classification (real/fake)
    ])

    return model

In [15]:
# Instantiate Generator and Discriminator
generator = build_generator(latent_dim)
discriminator = build_discriminator(input_shape)

# Compile Discriminator
discriminator.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
    metrics=['accuracy']
)


In [16]:
# Build GAN Model
discriminator.trainable = False
gan_input = layers.Input(shape=(latent_dim,))
fake_image = generator(gan_input)
gan_output = discriminator(fake_image)
gan = tf.keras.Model(gan_input, gan_output)
gan.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True)
)

In [18]:
gan.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 sequential (Sequential)     (None, 128, 1000, 1)      5149825   
                                                                 
 sequential_1 (Sequential)   (None, 1)                 268546561 
                                                                 
Total params: 273,696,386
Trainable params: 5,148,865
Non-trainable params: 268,547,521
_________________________________________________________________


In [19]:
# Training Function
def train_gan(generator, discriminator, gan, data, epochs, batch_size):
    half_batch = batch_size // 2
    for epoch in range(epochs):
        # Train Discriminator
        idx = np.random.randint(0, data.shape[0], half_batch)
        real_images = tf.convert_to_tensor(data[idx], dtype=tf.float32)
        real_labels = np.ones((half_batch, 1))

        noise = np.random.normal(0, 1, (half_batch, latent_dim))
        fake_images = generator.predict(noise)
        fake_labels = np.zeros((half_batch, 1))

        d_loss_real = discriminator.train_on_batch(real_images, real_labels)
        d_loss_fake = discriminator.train_on_batch(fake_images, fake_labels)

        # Train Generator
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        misleading_labels = np.ones((batch_size, 1))
        g_loss = gan.train_on_batch(noise, misleading_labels)

        # Log losses
        if epoch % 10 == 0:
            print(f"Epoch {epoch}/{epochs} | D Loss: {d_loss_real[0]+d_loss_fake[0]:.4f} | G Loss: {g_loss:.4f}")


In [20]:
# Train the GAN
train_gan(generator, discriminator, gan, mel_spectrograms, epochs=50, batch_size=64)

1/1 [==============================] - 1s 923ms/step
Epoch 0/50 | D Loss: 1.3933 | G Loss: 0.0004
1/1 [==============================] - 1s 652ms/step
Epoch 10/50 | D Loss: 0.0000 | G Loss: 35.8930
1/1 [==============================] - 1s 597ms/step
Epoch 20/50 | D Loss: 0.0000 | G Loss: 72.7631
1/1 [==============================] - 1s 578ms/step
Epoch 30/50 | D Loss: 0.0000 | G Loss: 1.8206
1/1 [==============================] - 1s 631ms/step
Epoch 40/50 | D Loss: 0.0000 | G Loss: 0.0000
1/1 [==============================] - 1s 590ms/step


In [26]:
import numpy as np
import librosa
import soundfile as sf

sr = 22050

# Generate New Audio
def generate_audio(generator, latent_dim, sr=22050):
    noise = np.random.normal(0, 1, (1, latent_dim))
    generated_mel = generator.predict(noise)
    generated_mel = generated_mel.squeeze()  # Remove extra dimensions

    # Ensure the Mel spectrogram is in the correct floating-point format
    generated_mel = np.clip(generated_mel, 0, 1)  # Ensure values are within range
    generated_mel = generated_mel.astype(np.float32)  # Convert to float32 for librosa

    # Convert Mel-Spectrogram back to audio
    mel_spectrogram = librosa.feature.inverse.mel_to_audio(generated_mel, sr=sr)
    return mel_spectrogram

# Example: Generate audio
generated_audio = generate_audio(generator, latent_dim)

# Save the generated audio as a .wav file using soundfile
sf.write('generated_audio.wav', generated_audio, sr)

1/1 [==============================] - 0s 34ms/step


In [27]:
# Save the generator model
generator.save('generator_model.h5')

# Save the discriminator model
discriminator.save('discriminator_model.h5')

# Save the GAN model (optional)
gan.save('gan_model.h5')

print("Models saved successfully.")

Models saved successfully.
